## Import

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn

import pandas as pd

import os

import numpy as np

from skorch import NeuralNetRegressor
from skorch.callbacks import EarlyStopping, Checkpoint, LRScheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from skorch.helper import predefined_split
from skorch.dataset import Dataset

In [30]:
def create_scaled_data_by_col(df, min_max_cols, normalize_cols, y_cols, col_name, col):
    if col_name in min_max_cols:
        min_max_cols.remove(col_name)
    if col_name in normalize_cols:
        normalize_cols.remove(col_name)

    db = df[df[col_name] == col]
    db = db.drop(columns=[col_name])

    x_min_max = db[min_max_cols].values.astype(np.float32)
    x_normalize = db[normalize_cols].values.astype(np.float32)
    y = db[y_cols].values.reshape(-1, len(y_cols)).astype(np.float32)

    min_max_scaler = MinMaxScaler().fit(x_min_max)
    normalize_scaler = StandardScaler().fit(x_normalize)
    y_scaler = StandardScaler().fit(y)

    x_min_max = min_max_scaler.transform(x_min_max)
    x_normalize = normalize_scaler.transform(x_normalize)
    y_final = y_scaler.transform(y)

    db[min_max_cols] = x_min_max
    db[normalize_cols] = x_normalize
    db[y_cols] = y_final
    

    return (db, min_max_scaler, normalize_scaler, y_scaler)



## Loading Data and Scaling

In [31]:
data_dir = 'data/'
df = pd.read_csv(os.path.join(data_dir, 'train_data.csv'))
display(df.head())

,store_nbr,family,sales,onpromotion,city,state,store_type,cluster,oil,h_type_nat,...,dow_avg_sales,dow_rolling_3_sales,dow_rolling_7_sales,dow_avg_transactions,dow_rolling_3_transactions,dow_rolling_7_transactions,rolling_7_sales,rolling_14_sales,rolling_7_transactions,rolling_14_transactions
0,1,0,7.0,0,0,0,0,13,53.9,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,0.0,0,0,0,0,13,53.9,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2,7.0,1,0,0,0,13,53.9,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,3,2399.0,28,0,0,0,13,53.9,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,4,1.0,0,0,0,0,13,53.9,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
min_max_cols = ['store_nbr', 'city', 'state', 'store_type', 'cluster', 'h_type_nat', 'h_description_nat', 'h_transferred_nat', 'h_type_loc', 'h_description_loc', 'h_transferred_loc', 'month', 'day', 'day_of_week']
normalize_cols = ['onpromotion', 'oil', 'dow_avg_sales', 'dow_rolling_3_sales', 'rolling_7_sales', 'rolling_14_sales', 'dow_avg_transactions', 'dow_rolling_3_transactions', 'rolling_7_transactions', 'rolling_14_transactions']
x_cols = min_max_cols + normalize_cols
y_cols = ['sales']
split_col = 'family'

print(min_max_cols)
print(df.columns)

final_run = False

if final_run:
    train_df = df
else:
    rows_after = (df['year'] >= 2017) & (df['month'] >= 8) & (df['day'] >= 1)
    rows_before = ~rows_after

    train_df = df[rows_before]
    val_df = df[rows_after]


train_df_by_cluster = {}
scaler_x_by_cluster = {}
scaler_y_by_cluster = {}

for cluster in df[split_col].unique():
    cluster_df, cluster_min_max_scaler, cluster_normalize_scaler, cluster_y_scaler = create_scaled_data_by_col(train_df, min_max_cols, normalize_cols, y_cols, split_col, cluster)
    train_df_by_cluster[cluster] = cluster_df
    scaler_x_by_cluster[cluster] = (cluster_min_max_scaler, cluster_normalize_scaler)
    scaler_y_by_cluster[cluster] = cluster_y_scaler

if not final_run:
    val_df_by_cluster = {}

    for cluster in df[split_col].unique():
        val_cluster_min_max_scaler, val_cluster_normalize_scaler = scaler_x_by_cluster[cluster]
        val_cluster_y_scaler = scaler_y_by_cluster[cluster]

        val_cluster_df = val_df[val_df[split_col] == cluster]
        val_cluster_df = val_cluster_df.drop(columns=split_col)

        val_cluster_x_min_max = val_cluster_df[min_max_cols].values.astype(np.float32)
        val_cluster_x_normalize = val_cluster_df[normalize_cols].values.astype(np.float32)
        val_cluster_y = val_cluster_df[y_cols].values.reshape(-1, len(y_cols)).astype(np.float32)

        val_cluster_x_min_max = val_cluster_min_max_scaler.transform(val_cluster_x_min_max)
        val_cluster_x_normalize = val_cluster_normalize_scaler.transform(val_cluster_x_normalize)
        val_cluster_y = val_cluster_y_scaler.transform(val_cluster_y)

        val_cluster_df[min_max_cols] = val_cluster_x_min_max
        val_cluster_df[normalize_cols] = val_cluster_x_normalize
        val_cluster_df[y_cols] = val_cluster_y

        val_df_by_cluster[cluster] = val_cluster_df

['store_nbr', 'city', 'state', 'store_type', 'cluster', 'h_type_nat', 'h_description_nat', 'h_transferred_nat', 'h_type_loc', 'h_description_loc', 'h_transferred_loc', 'month', 'day', 'day_of_week']
Index(['store_nbr', 'family', 'sales', 'onpromotion', 'city', 'state',
       'store_type', 'cluster', 'oil', 'h_type_nat', 'h_description_nat',
       'h_transferred_nat', 'h_type_loc', 'h_description_loc',
       'h_transferred_loc', 'year', 'month', 'day', 'day_of_week',
       'dow_avg_sales', 'dow_rolling_3_sales', 'dow_rolling_7_sales',
       'dow_avg_transactions', 'dow_rolling_3_transactions',
       'dow_rolling_7_transactions', 'rolling_7_sales', 'rolling_14_sales',
       'rolling_7_transactions', 'rolling_14_transactions'],
      dtype='object')


## Neural Network

In [33]:
import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, out_dim, hidden_dim=64, num_hidden_layers=2):
        super(NeuralNetwork, self).__init__()

        self.act = nn.ReLU()
        self.head = nn.Linear(input_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, out_dim)

        layers = []
        for i in range(num_hidden_layers):
            layers.append(nn.Linear(hidden_dim, hidden_dim ))

        self.stem = nn.Sequential(*layers)


    def forward(self, x):
        x = self.act(self.head(x))

        for layer in self.stem:
            x =  x + self.act(layer(x))

        return self.output(x)

## NN Training

In [34]:
net_by_cluster = {}
train_params = {
                "criterion": nn.L1Loss,
                "optimizer": torch.optim.AdamW,
                "optimizer__weight_decay": 1e-8,
                #'train_split' : None,
                #"train_split": predefined_split(Dataset(val_x, val_y)),
                "lr": 0.001,
                "batch_size": 128,
                "max_epochs": 1000,
                "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
                "iterator_train__shuffle": False,
                "iterator_train__num_workers": 2,
                "iterator_train__pin_memory": True,
                "iterator_valid__shuffle": False,
                "iterator_valid__num_workers": 2,
                "iterator_valid__pin_memory": True,
                "verbose": 2,
        }

net_params = {
    'input_dim': len(x_cols),
    'out_dim': 1,
    'hidden_dim': 200,
    'num_hidden_layers': 6,
    }

In [35]:
for cluster in df[split_col].unique():
    train_df = train_df_by_cluster[cluster]
    train_x = train_df[x_cols].values.astype(np.float32)
    train_y = train_df[y_cols].values.reshape(-1, len(y_cols)).astype(np.float32)

    if not final_run:
        val_df = val_df_by_cluster[cluster]
        train_params['train_split'] = predefined_split(Dataset(val_df[x_cols].values.astype(np.float32), val_df[y_cols].values.reshape(-1, len(y_cols)).astype(np.float32)))
    else:
        train_params['train_split'] = None

    callbacks = [EarlyStopping(patience=15, threshold=0.001, threshold_mode='abs', monitor='valid_loss', lower_is_better=True),
            Checkpoint(monitor='valid_loss_best', f_params=f'sales_forecaster_{cluster}.pt', dirname='models/'),
            LRScheduler(policy=ReduceLROnPlateau, monitor='valid_loss', factor=0.5, patience=5, threshold=0.001, threshold_mode='abs', mode='min', verbose=True)
            ]

    train_params['callbacks'] = callbacks

    net = NeuralNetRegressor(NeuralNetwork(**net_params), **train_params)

    net.fit(train_x, train_y)
    net_by_cluster[cluster] = net

/home/codygrogan/.local/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


  epoch    train_loss    valid_loss    cp     dur
-------  ------------  ------------  ----  ------
      1        0.5912        0.5464     +  0.3463
      2        0.5566        0.4985     +  0.3242
      3        0.5363        0.4945     +  0.3751
      4        0.5159        0.4933     +  0.3629
      5        0.5120        0.4913     +  0.3543
      6        0.5070        0.4904     +  0.3464
      7        0.5051        0.4915        0.3461
      8        0.5011        0.4900     +  0.3458
      9        0.4967        0.4890     +  0.3372
     10        0.4915        0.4894        0.3510
     11        0.4899        0.4901        0.3383
     12        0.4880        0.4913        0.3431
     13        0.4856        0.4920        0.3521
     14        0.4820        0.4910        0.3563
     15        0.4812        0.4932        0.3492
     16        0.4935        0.4920        0.3380
     17        0.4854        0.4880     +  0.3268
     18        0.4797        0.4922        0.3219


/home/codygrogan/.local/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


  epoch    train_loss    valid_loss    cp     dur
-------  ------------  ------------  ----  ------
      1        0.4134        0.2961     +  0.3388
      2        0.3396        0.2766     +  0.3694
      3        0.3397        0.2933        0.3469
      4        0.3326        0.2654     +  0.3471
      5        0.3476        0.2785        0.3556
      6        0.3306        0.2613     +  0.3443
      7        0.3159        0.2613        0.3545
      8        0.3199        0.2813        0.3557
      9        0.3175        0.2753        0.3347
     10        0.3212        0.2719        0.3574
     11        0.3141        0.2807        0.3468
     12        0.3207        0.2677        0.3302
     13        0.3057        0.2656        0.3683
     14        0.3034        0.2626        0.3508
     15        0.3011        0.2640        0.3547
     16        0.3025        0.2593     +  0.3469
     17        0.3022        0.2604        0.3601
     18        0.3016        0.2691        0.3449


/home/codygrogan/.local/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


...

  epoch    train_loss    valid_loss    cp     dur
-------  ------------  ------------  ----  ------
      1        0.3102        0.1526     +  0.3445
      2        0.2425        0.1538        0.3379
      3        0.2323        0.1972        0.3533
      4        0.2058        0.1238     +  0.3295
      5        0.1566        0.1209     +  0.3431
      6        0.1579        0.1239        0.3485
      7        0.1735        0.1193     +  0.3550
      8        0.1408        0.1097     +  0.3389
      9        0.1373        0.1084     +  0.3496
     10        0.1354        0.1198        0.3276
     11        0.1506        0.1268        0.3355
     12        0.1369        0.1330        0.3429
     13        0.1346        0.1178        0.3357
     14        0.1343        0.1197        0.3314
     15        0.1364        0.1105        0.3421
     16        0.1278        0.1078     +  0.3401
     17        0.1358        0.1021     +  0.3468
     18        0.1299        0.1032        0.3551


/home/codygrogan/.local/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


  epoch    train_loss    valid_loss    cp     dur
-------  ------------  ------------  ----  ------
      1        0.3880        2.5490     +  0.3380
      2        0.2265        2.5897        0.3418
      3        0.2335        2.4027     +  0.3455
      4        0.2053        2.3321     +  0.3526
      5        0.1870        2.4104        0.3453
      6        0.1878        2.3122     +  0.3409
      7        0.1789        2.2431     +  0.3390
      8        0.1780        2.2129     +  0.3280
      9        0.1790        2.2727        0.3593
     10        0.1799        2.2427        0.3491
     11        0.1791        2.0793     +  0.3438
     12        0.1631        2.0190     +  0.3257
     13        0.1617        1.8733     +  0.3512
     14        0.1600        2.0282        0.3464
     15        0.1571        2.1355        0.3392
     16        0.1585        2.1293        0.3441
     17        0.1566        2.1644        0.3238
     18        0.1562        2.0221        0.3464


/home/codygrogan/.local/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


  epoch    train_loss    valid_loss    cp     dur
-------  ------------  ------------  ----  ------
      1        0.3117        0.2093     +  0.3527
      2        0.2545        0.1590     +  0.3392
      3        0.2583        0.2420        0.3922
      4        0.2496        0.2012        0.3549
      5        0.2260        0.1828        0.3543
      6        0.2260        0.1714        0.3549
      7        0.2231        0.1684        0.3445
      8        0.2130        0.1606        0.3447
      9        0.2087        0.1691        0.3488
     10        0.2049        0.1638        0.3323
     11        0.2049        0.1598        0.3446
     12        0.2004        0.1586     +  0.3498
     13        0.2017        0.1608        0.3463
     14        0.1963        0.1581     +  0.3662
     15        0.1983        0.1542     +  0.3365
     16        0.1967        0.1534     +  0.3417
     17        0.1944        0.1530     +  0.3358
     18        0.1924        0.1531        0.3430


# Load Nets from Checkpoints

In [36]:
for cluster in df[split_col].unique():
    net = NeuralNetRegressor(NeuralNetwork(**net_params), **train_params)
    net.initialize()
    net.load_params(f_params=f'models/sales_forecaster_{cluster}.pt')
    net_by_cluster[cluster] = net

## Random Forest

In [37]:
from sklearn.ensemble import RandomForestRegressor
cluster_rfs = {}

for cluster in df[split_col].unique():
    train_df = train_df_by_cluster[cluster]

    train_x = train_df[x_cols].values.astype(np.float32)
    train_y = train_df[y_cols].values.reshape(-1, len(y_cols)).astype(np.float32)

    rf = RandomForestRegressor(n_estimators=100, max_depth=20, random_state=42, n_jobs=4)
    rf.fit(train_x, train_y.squeeze())

    cluster_rfs[cluster] = rf

## XGBoost

In [18]:
import xgboost as xgb
cluster_xgb = {}
for cluster in df[split_col].unique():
    train_x = train_x_by_cluster[cluster]
    train_y = train_y_by_cluster[cluster]

    xgb_model = xgb.XGBRegressor(n_estimators=1000, max_depth=12, learning_rate=0.001, random_state=42, n_jobs=2)
    xgb_model.fit(train_x, train_y.squeeze())

    cluster_xgb[cluster] = xgb_model

## Predict on Training Data

In [27]:
net_train_preds = []
rf_train_preds = []

for cluster in df[split_col].unique():
    train_df = train_df_by_cluster[cluster]

    train_x = train_df[x_cols].values.astype(np.float32)
    train_y = train_df[y_cols].values.reshape(-1, len(y_cols)).astype(np.float32)
    y_scaler = scaler_y_by_cluster[cluster]

    net = net_by_cluster[cluster]
    rf = cluster_rfs[cluster]

    net_preds = net.predict(train_x)
    rf_preds = rf.predict(train_x)

    train_df['sales_nn'] = net_preds
    train_df['sales_rf'] = rf_preds

## Validation Loss Evaluation

In [38]:
def rmsle(y_true, y_pred):
    return np.sqrt(np.mean(np.square(np.log1p(y_true) - np.log1p(y_pred))))

rf_preds = []
net_preds = []
xgb_preds = []
val_y_true = []

for cluster in df[split_col].unique():
    val_cluster_df = val_df_by_cluster[cluster]
    val_x = val_cluster_df[x_cols].values.astype(np.float32)
    val_y = val_cluster_df[y_cols].values.reshape(-1, len(y_cols)).astype(np.float32)

    rf = cluster_rfs[cluster]
    net = net_by_cluster[cluster]

    rf_preds.append(scaler_y_by_cluster[cluster].inverse_transform(rf.predict(val_x).reshape(-1, 1)))
    net_preds.append(scaler_y_by_cluster[cluster].inverse_transform(net.predict(val_x).reshape(-1, 1)).clip(0))    
    #xgb_preds.append(scaler_y_by_cluster[cluster].inverse_transform(cluster_xgb[cluster].predict(val_x).reshape(-1, 1)))
    val_y_true.append(scaler_y_by_cluster[cluster].inverse_transform(val_y))

rf_preds = np.concatenate(rf_preds)
net_preds = np.concatenate(net_preds)
#xgb_preds = np.concatenate(xgb_preds)
val_y_true = np.concatenate(val_y_true)

print(f'RF RMSLE: {rmsle(val_y_true, rf_preds)}')
#print(f'XGB RMSLE: {rmsle(val_y_true, xgb_preds)}')
print(f'NN RMSLE: {rmsle(val_y_true, net_preds)}')

RF RMSLE: 0.435911921206087
NN RMSLE: 0.4095858931541443


## Loading Test Data

In [12]:
test_df = pd.read_csv(os.path.join(data_dir, 'test_data.csv'), index_col=0)
display(test_df.head())

test_x_by_cluster = {}
test_id_by_cluster = {}

for cluster in df[split_col].unique():
    test_cluster_min_max_scaler, test_cluster_normalize_scaler = scaler_x_by_cluster[cluster]
    test_cluster_y_scaler = scaler_y_by_cluster[cluster]

    test_cluster_x_df = test_df[test_df[split_col] == cluster]
    test_cluster_x_df = test_cluster_x_df.drop(columns=split_col)

    test_cluster_x_min_max = test_cluster_x_df[min_max_cols].values.astype(np.float32)
    test_cluster_x_normalize = test_cluster_x_df[normalize_cols].values.astype(np.float32)

    test_cluster_x_min_max = test_cluster_min_max_scaler.transform(test_cluster_x_min_max)
    test_cluster_x_normalize = test_cluster_normalize_scaler.transform(test_cluster_x_normalize)

    test_x_by_cluster[cluster] = np.concatenate([test_cluster_x_min_max, test_cluster_x_normalize], axis=1)
    test_id_by_cluster[cluster] = test_cluster_x_df.index


test_preds_dfs = []

for cluster in df[split_col].unique():
    test_x = test_x_by_cluster[cluster]
    id = test_id_by_cluster[cluster]
    #rf = cluster_rfs[cluster]

    #pred_rf = scaler_y_by_cluster[cluster].inverse_transform(rf.predict(test_x).reshape(-1, 1))
    #pred_xgb = scaler_y_by_cluster[cluster].inverse_transform(cluster_xgb[cluster].predict(test_x).reshape(-1, 1))
    pred_nn = scaler_y_by_cluster[cluster].inverse_transform(net_by_cluster[cluster].predict(test_x).reshape(-1, 1)).clip(0)
    
    cluster_df = pd.DataFrame(np.concatenate([pred_nn], axis=1), index=id, columns=['sales_nn'])
    #cluster_df = pd.DataFrame(np.concatenate([pred_rf, pred_nn], axis=1), index=id, columns=['sales_rf', 'sales_nn'])

    test_preds_dfs.append(cluster_df)

test_preds_df = pd.concat(test_preds_dfs)

test_df = test_df.merge(test_preds_df, on='id', how='left')

sub_df_nn = test_df[['sales_nn']]
#sub_df_rf = test_df[['sales_rf']]
#sub_df_xgb = test_df[['sales_xgb']]

#sub_df_rf = sub_df_rf.rename(columns={'sales_rf': 'sales'})
#sub_df_xgb = sub_df_xgb.rename(columns={'sales_xgb': 'sales'})
sub_df_nn = sub_df_nn.rename(columns={'sales_nn': 'sales'})


display(sub_df_nn.head())
#display(sub_df_rf.head())
#display(sub_df_xgb.head())

sub_df_nn.to_csv('data/submission_nn.csv')
#sub_df_xgb.to_csv('data/submission_xgb.csv')
#sub_df_rf.to_csv('data/submission_rf.csv')

,store_nbr,family,onpromotion,city,state,store_type,cluster,oil,h_type,h_locale,...,day,day_of_week,dow_avg_sales,rolling_7_sales,rolling_14_sales,rolling_30_sales,dow_avg_transactions,rolling_7_transactions,rolling_14_transactions,rolling_30_transactions
id,,,,,,,,,,,,,,,,,,,,,
3000888,1,0,0,0,0,0,13,46.8,1,1,...,16,2,3.531381,3.428571,3.928571,4.133333,1863.393305,1888.857143,1863.857143,1859.3
3000889,1,1,0,0,0,0,13,46.8,1,1,...,16,2,0.000000,0.000000,0.000000,0.000000,1863.393305,1888.857143,1863.857143,1859.3
3000890,1,2,2,0,0,0,13,46.8,1,1,...,16,2,2.615063,3.857143,4.428571,3.700000,1863.393305,1888.857143,1863.857143,1859.3
3000891,1,3,20,0,0,0,13,46.8,1,1,...,16,2,1845.485356,2456.142857,2471.571429,2504.466667,1863.393305,1888.857143,1863.857143,1859.3
3000892,1,4,0,0,0,0,13,46.8,1,1,...,16,2,0.167364,0.000000,0.571429,0.733333,1863.393305,1888.857143,1863.857143,1859.3


,sales
id,
3000888,3.804415
3000889,0.000565
3000890,5.299076
3000891,2225.056641
3000892,0.000060
